In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

# import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
plt.close('all')
plt.rc('text', usetex=False)
# plt.rc('font', family='serif')
LEGEND_FONT_SIZE = 12
AXIS_FONT_SIZE = 10
TICKS_FONT_SIZE = 10
TITLE_FONT_SIZE = 12
linewidth=2.0
lw=1.2
current_palette = sns.color_palette()

matplotlib.rcParams["xtick.labelsize"] = TICKS_FONT_SIZE

In [ ]:
data = pd.read_csv("listings.csv", low_memory=False) 

In [ ]:
len(data)

In [ ]:
print(", ".join(data.columns))

In [ ]:
kept_columns = ["host_is_superhost", 'latitude','longitude', "accommodates", 
                "bathrooms", "bedrooms", "price", "number_of_reviews",
                "review_scores_rating", "instant_bookable"]

response = "price"
covariates = [c for c in kept_columns if c != response]

In [ ]:
data = data[kept_columns]

In [ ]:
data.head()

In [ ]:
## for each column, check the percentage of NAs

nas = np.zeros(len(kept_columns))
for i, k in enumerate(kept_columns):
    nas[i] = data[k].isna().sum() / len(data)
    
plt.bar(kept_columns, nas)
plt.xticks(rotation=90)
plt.show()

In [ ]:
## and drop the NAs
data = data.dropna()

In [ ]:
#convert prices to float
def convert_to_float(value):
    # remove ","
    string = value[1:].replace(",","")
    return float(string)

data['price'] = data['price'].apply(lambda x: convert_to_float(x))

In [ ]:
numeric_variables = ["accommodates", "bathrooms", "bedrooms", "price", 
                     "number_of_reviews", "review_scores_rating", "instant_bookable"]

in_log_scale = ["number_of_reviews"]

fig, axes = plt.subplots(2, 4, figsize=(12, 4))
axes = axes.flat

for i, k in enumerate(numeric_variables):
    if k == "price":
        axes[i].hist(data[k].values, range=(0, 800), color="orange")
    elif k in in_log_scale:
        axes[i].hist(np.log(data[k].values+1))
        axes[i].set_xlabel("log scale")
    else:
        axes[i].hist(data[k].values)
    axes[i].set_title(" ".join(k.split("_")), fontsize=TITLE_FONT_SIZE)
    
    
    
plt.tight_layout()
plt.show()

# plot neighbourhoods
plt.figure(figsize=(10,10))
sns.scatterplot(x='longitude', y='latitude', hue='neighbourhood',s=20, data=data)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
import geopandas as gpd

In [ ]:
#load geojson
geodf = gpd.read_file('neighbourhoods.geojson')

## Find correspondence between locations and geojson neigh

In [ ]:
#Create a point of each Airbnb location, and enable the "data" dataframe into a geopandas dataframe
datageom = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

#Now, do a spatial join... This code here runs an intersect analysis to find which neighborhood the Airbnb location is in
joined = gpd.sjoin(geodf, datageom, how='inner', op='intersects')
#N.B. some data are removed if not in any polygon

#Rename the column. 
joined.rename(columns={'neighbourhood_left':'neighbourhood'}, inplace=True)

#Drop the second geometry column
joined.drop(columns='geometry', inplace=True)

In [ ]:
# plot neighbourhoods
fig,ax1 = plt.subplots(1,1, figsize=(10,10))
base = geodf.plot(color='white',alpha=0.5, edgecolor='black', ax=ax1)
sns.scatterplot(x='longitude', y='latitude', hue='neighbourhood',s=20, data=joined)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
#Create a count of each neighborhood
nb_join_count = joined.groupby('neighbourhood').agg('count').reset_index()

#Get the "true count". Join this data to the original neighborhoods geometry 
true_count = geodf.merge(nb_join_count, on='neighbourhood')

#Lets plot number of observations in each area
fig,ax = plt.subplots(1,1, figsize=(10,10))

#log scale
true_count['price'] = np.log(true_count['price'])
base = geodf.plot(color='white', edgecolor='black', ax=ax)
true_count.plot(column='price',cmap='viridis', ax=base, legend=True)
plt.title('Number of Airbnb listings by Neighborhood in Amsterdam')
plt.show()

In [ ]:
plt.hist(np.log(joined.number_of_reviews + 1))
joined.loc[:, "number_of_reviews"] = np.log(joined.loc[:, "number_of_reviews"].values + 1) 

In [ ]:
from scipy.stats import boxcox, probplot, norm

In [ ]:
x_t, param = boxcox(joined.review_scores_rating)
print(param)
plt.hist(x_t)
plt.show()
joined.loc[:, "review_scores_rating"] = x_t

# Standardize numerical covariates

In [ ]:
standardized = joined

for col in ["accommodates", "bathrooms", "bedrooms", "number_of_reviews", "review_scores_rating"]:
    mean = joined.loc[:, col].mean()
    std = joined.loc[:, col].std()
    standardized.loc[:, col] -= mean
    standardized.loc[:, col] /= std

In [ ]:
standardized.corr()

In [ ]:
plt.hist(standardized.review_scores_rating)

In [ ]:
# convert data to list of lists
grouped_price = standardized.groupby(['neighbourhood']).apply(lambda x: x['price'].values.tolist())
grouped_cov = standardized.groupby(['neighbourhood']).apply(lambda x: x[covariates].values.tolist())
neigh = grouped_cov.index.values

price = []
cov = []
low = 1
up = 1000
for index, value in grouped_price.items():
    p = np.array(value)
    bound_indices = np.where(np.logical_and(p<up, p>low))[0]
    price.append(p[bound_indices])
    cov.append(pd.DataFrame(data=np.array(grouped_cov[index])[bound_indices],
                            columns=covariates))

In [ ]:
# remove empty neigh from geodf
drop_ind = [ i for i in range(geodf.shape[0]) if geodf['neighbourhood'].iloc[i] not in neigh]
geodf.drop(drop_ind, inplace=True, axis=0)
geodf = geodf.reset_index()

In [ ]:
# build W
neigh_map = {k: i for i,k in enumerate(neigh) }

n = len(neigh)
W = np.zeros((n, n))
for index, value in geodf.iterrows():  
    # get 'not disjoint' countries
    neighbors = geodf[~geodf.geometry.disjoint(value.geometry)].neighbourhood.tolist()
    # remove own name from the list
    neighbors = [ name for name in neighbors if value.neighbourhood != name ]
    # get indices
    neighbors_indices = [i for i in range(n) if neigh[i] in neighbors]
    # W is 1 for neighbors
    W[neigh_map[value.neighbourhood], neighbors_indices] = 1   

In [ ]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
matprint(W)

In [ ]:
j=1
fig,ax1 = plt.subplots(1,1, figsize=(30,30))

ax2 = geodf.plot(color='white',alpha=0.5, edgecolor='black', ax=ax1)
ax3 = geodf.iloc[np.where(geodf['neighbourhood']==neigh[j])[0],:].plot(color='blue',
                                                                       alpha=0.5, edgecolor='black', ax=ax2)

geodf.iloc[np.where(geodf['neighbourhood'].isin(neigh[np.where(W[j,:])[0]]))[0],:].plot(
    color='red',alpha=0.5, edgecolor='black', ax=ax3)
geodf.apply(lambda x: ax3.annotate(s=neigh_map[x.neighbourhood], color='black', 
                                   xy=x.geometry.centroid.coords[0],ha='center'), axis=1)
plt.show()

In [ ]:
# 3 e 18 non sono vicini
W[3,18] = 0
W[18,3] = 0

In [ ]:
for i in range(len(cov)):
    if len(price[i]) != cov[i].shape[0]:
        print("error in ", i)
        print("datas: ", len(price[i]), " covariates", cov[i].shape[0])

In [ ]:
# save W and l
import pickle

d={'y':price, 'G':W, 'neigh_names':neigh, 'cov': cov}
with open('../../data/amsterdam.pkl', 'wb') as f:
    pickle.dump(d, f)

In [ ]:
import contextily as ctx
# plot densità stimate
indices = [0, 9, 19, 14]
palette = sns.color_palette()

geodf = geodf.to_crs(epsg=3857)

fig,ax = plt.subplots(1,1, figsize=(20,20))

ax = geodf.plot(fc=(0,0,0,0.1), edgecolor='black', ax=ax)

for i in range(len(indices)):
    ax = geodf.iloc[np.where(geodf['neighbourhood']==neigh[indices[i]])[0],:].plot(color=palette[i],
                                                                       alpha=0.9, edgecolor='black', ax=ax)
ctx.add_basemap(ax, alpha=0.5)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_axis_off()
#ax.set_aspect('auto')

#geodf.apply(lambda x: ax.annotate(s=x.neighbourhood, color='black', 
#                                   xy=x.geometry.centroid.coords[0],ha='center'), axis=1)
#plt.show()
plt.savefig('dens_estimates_neigh.pdf')